# Data Cleaning

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from config import state_abbrev, reverse_state_abbrev, state_codes

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

## US Air Quality Index (AQI) Data
By state/county, with FIPS data for choropleth plotting.

In [2]:
# Read in and combine AQI data
county_aqi_df = pd.DataFrame()
for yr in range(1980,2018):
    aqi = pd.read_csv('raw_data/annual_aqi_by_county_{}.csv'.format(yr))
    county_aqi_df = county_aqi_df.append(aqi, ignore_index=True)

# Make state and county names lowercase
for y in ['State', 'County']:
    county_aqi_df[y] = county_aqi_df[y].apply(lambda x: x.lower())

county_aqi_df.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,alabama,autauga,1980,179,122,35,18,4,0,0,177,108,40,0,0,122,57,0,0
1,alabama,colbert,1980,274,127,45,63,39,0,0,200,165,56,0,0,0,274,0,0
2,alabama,jackson,1980,366,85,110,92,79,0,0,200,200,94,0,0,0,366,0,0
3,alabama,jefferson,1980,342,174,105,37,19,7,0,221,140,50,202,0,140,0,0,0
4,alabama,lauderdale,1980,274,120,58,77,19,0,0,200,139,56,0,0,0,274,0,0


In [3]:
# Get dataframe with state/county FIPS codes
fips = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')

# Parse FIPS county/state data
fips["County"] = fips['County Name/State Abbreviation'].apply(lambda x: x.split(",")[0].replace(" County", "").lower())
fips["State Abbreviation"] = fips['County Name/State Abbreviation'].apply(lambda x: x.split(", ")[-1])
fips["State"] = fips["State Abbreviation"].apply(lambda x: state_abbrev[x].lower())

# Pad FIPS codes with 0's, concat for combined FIPS code
fips['State FIPS Code'] = fips['State FIPS Code'].apply(lambda x: str(x).zfill(2))
fips['County FIPS Code'] = fips['County FIPS Code'].apply(lambda x: str(x).zfill(3))
fips['FIPS'] = fips['State FIPS Code'] + fips['County FIPS Code']

# Remove unused columns
fips = fips[['County','State','State Abbreviation','FIPS','State FIPS Code','County FIPS Code']]

fips.head()

,County,State,State Abbreviation,FIPS,State FIPS Code,County FIPS Code
0,autauga,alabama,AL,01001,01,001
1,baldwin,alabama,AL,01003,01,003
2,barbour,alabama,AL,01005,01,005
3,bibb,alabama,AL,01007,01,007
4,blount,alabama,AL,01009,01,009


In [56]:
# Combine with county AQI df
county_aqi_1980_2018 = county_aqi_df.merge(fips, how='inner', on=['State','County'])
county_aqi_1980_2018.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,...,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,State Abbreviation,FIPS,State FIPS Code,County FIPS Code
0,alabama,autauga,1980,179,122,35,18,4,0,0,...,0,0,122,57,0,0,AL,01001,01,001
1,alabama,autauga,1981,357,289,49,15,4,0,0,...,0,0,241,116,0,0,AL,01001,01,001
2,alabama,autauga,1982,245,203,36,5,0,1,0,...,0,0,166,79,0,0,AL,01001,01,001
3,alabama,autauga,1989,63,54,9,0,0,0,0,...,0,0,63,0,0,0,AL,01001,01,001
4,alabama,autauga,1990,266,183,64,18,1,0,0,...,0,0,266,0,0,0,AL,01001,01,001


In [57]:
# Add columns with percentage data for good/hazardous days
county_aqi_1980_2018['% Days Good'] = county_aqi_1980_2018["Good Days"] / county_aqi_1980_2018["Days with AQI"] * 100
county_aqi_1980_2018['% Days Moderate'] = county_aqi_1980_2018["Moderate Days"] / county_aqi_1980_2018["Days with AQI"] * 100
county_aqi_1980_2018['% Days Unhealthy for Sensitive Groups'] = county_aqi_1980_2018["Unhealthy for Sensitive Groups Days"] / county_aqi_1980_2018["Days with AQI"] * 100
county_aqi_1980_2018['% Days Unhealthy'] = county_aqi_1980_2018["Unhealthy Days"] / county_aqi_1980_2018["Days with AQI"] * 100
county_aqi_1980_2018['% Days Very Unhealthy'] = county_aqi_1980_2018["Very Unhealthy Days"] / county_aqi_1980_2018["Days with AQI"] * 100
county_aqi_1980_2018['% Days Hazardous'] = county_aqi_1980_2018["Hazardous Days"] / county_aqi_1980_2018["Days with AQI"] * 100
county_aqi_1980_2018.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,...,State Abbreviation,FIPS,State FIPS Code,County FIPS Code,% Days Good,% Days Moderate,% Days Unhealthy for Sensitive Groups,% Days Unhealthy,% Days Very Unhealthy,% Days Hazardous
0,alabama,autauga,1980,179,122,35,18,4,0,0,...,AL,01001,01,001,68.156425,19.553073,10.055866,2.234637,0.000000,0.0
1,alabama,autauga,1981,357,289,49,15,4,0,0,...,AL,01001,01,001,80.952381,13.725490,4.201681,1.120448,0.000000,0.0
2,alabama,autauga,1982,245,203,36,5,0,1,0,...,AL,01001,01,001,82.857143,14.693878,2.040816,0.000000,0.408163,0.0
3,alabama,autauga,1989,63,54,9,0,0,0,0,...,AL,01001,01,001,85.714286,14.285714,0.000000,0.000000,0.000000,0.0
4,alabama,autauga,1990,266,183,64,18,1,0,0,...,AL,01001,01,001,68.796992,24.060150,6.766917,0.375940,0.000000,0.0


In [58]:
# Export county aqi data to csv
county_aqi_1980_2018.to_csv("clean_data/county_aqi_1980_2018.csv")
print('Done.')

Done.


## US Cancer Data - by State

In [6]:
# Read in csvs as dataframes
df_cancer_1998_2008 = pd.read_csv("raw_data/state_yearly_monthly_cancer_statistics_1999_to_2008.csv")
df_cancer_2009_2014 = pd.read_csv("raw_data/state_yearly_monthly_cancer_statistics_2009_to_2014.csv")

# Remove lines without data
df_cancer_1998_2008 = df_cancer_1998_2008.loc[df_cancer_1998_2008["State"].isin(reverse_state_abbrev)]
df_cancer_2009_2014 = df_cancer_2009_2014.loc[df_cancer_2009_2014["State"].isin(reverse_state_abbrev)]

# Combine dataframes
df_cancer_1998_2014 = df_cancer_1998_2008.append(df_cancer_2009_2014, ignore_index=True)

# Add column with state abbreviation for making choropleth
df_cancer_1998_2014["State Abbreviation"] = df_cancer_1998_2014["State"].apply(lambda x: reverse_state_abbrev[x])

# Change numeric dtypes
df_cancer_1998_2014[['State Code', 'Year', 'Year Code', 'Count']\
                   ] = df_cancer_1998_2014[['State Code', 'Year', 'Year Code', 'Count']].astype(int)

state_cancer_1998_2014 = df_cancer_1998_2014[['State','State Abbreviation','Year','Year Code','Cancer Sites','Cancer Sites Code','Count']]
state_cancer_1998_2014.head()

,State,State Abbreviation,Year,Year Code,Cancer Sites,Cancer Sites Code,Count
0,Connecticut,CT,1999,1999,All Invasive Cancer Sites Combined,0,18694
1,Connecticut,CT,2000,2000,All Invasive Cancer Sites Combined,0,19200
2,Connecticut,CT,2001,2001,All Invasive Cancer Sites Combined,0,19626
3,Connecticut,CT,2002,2002,All Invasive Cancer Sites Combined,0,19570
4,Connecticut,CT,2003,2003,All Invasive Cancer Sites Combined,0,19516


In [7]:
# Export state cancer data to csv
state_cancer_1998_2014.to_csv("clean_data/state_cancer_1998_2014.csv")
print('Done.')

Done.


In [8]:
# Group data by state
cancer_by_state = state_cancer_1998_2014.groupby(['State Abbreviation','State'])[['Count']].sum()
# cancer_by_state = cancer_by_state.reset_index('State Abbreviation')
cancer_by_state.head()

,,Count
State Abbreviation,State,
AK,Alaska,125735
AL,Alabama,1215322
AR,Arkansas,668683
AZ,Arizona,1356728
CA,California,7956919


## US Cancer Data - by County

In [104]:
# What year is this data for??

# Read in county data for cancer incidences
csv = "Choropleth Testing/Resources/county_cancer_data.csv"
county_cancer_data = pd.read_csv(csv, encoding='latin-1')
county_cancer_data = county_cancer_data.rename(columns = {"County": "Geography"})

# Parse geographic info into county and state
county_cancer_data['County'] = county_cancer_data['Geography'].apply(lambda x: x.split(', ')[0])
county_cancer_data['State'] = county_cancer_data['Geography'].apply(lambda x: x.split(', ')[-1].split('(')[0])

# Remove values that don't contain state data
county_cancer_data = county_cancer_data.loc[county_cancer_data['State'] != 'US ']
county_cancer_data = county_cancer_data.loc[county_cancer_data['State'] != 'Puerto Rico']

# Remove null datapoints
county_cancer_data = county_cancer_data.loc[county_cancer_data['Average Annual Count'] != '¶ ']
county_cancer_data = county_cancer_data.loc[county_cancer_data['Average Annual Count'] != '¶¶']
county_cancer_data = county_cancer_data.loc[county_cancer_data['Average Annual Count'] != '¶']
county_cancer_data = county_cancer_data.loc[county_cancer_data['Met Healthy People Objective of ***?'] != '<font color=grey>*</font>']

# Add column for state abbreviation
county_cancer_data['State Abbreviation'] = county_cancer_data['State'].apply(lambda x: reverse_state_abbrev[x])

county_cancer_data.head()

,Geography,FIPS,Met Healthy People Objective of ***?,"Age-Adjusted Incidence Rate() - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend () in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,County,State,State Abbreviation
1,"Union County, Florida(6,10)",12125,***,215.1,183.2,251.1,38,stable,-2.5,-24,25,Union County,Florida,FL
2,"Owsley County, Kentucky(7,8)",21189,***,179.7,134.8,236.3,11,rising,4.3,0.9,7.7,Owsley County,Kentucky,KY
3,"McCreary County, Kentucky(7,8)",21147,***,154.1,130.5,180.9,32,stable,-0.7,-3,1.7,McCreary County,Kentucky,KY
4,"North Slope Borough, Alaska(6,10)",2185,***,150.8,97,220.1,7,stable,-0.2,-36.8,57.8,North Slope Borough,Alaska,AK
5,"Powell County, Kentucky(7,8)",21197,***,140.9,115.2,171,22,stable,0.4,-3.3,4.3,Powell County,Kentucky,KY


In [ ]:
# Export county cancer data to csv
# county_cancer_data_years = [???]
# county_cancer_data.to_csv("clean_data/county_cancer_d.csv")
# print('Done.')

## County Population Data

In [51]:
# Read in census data with county population estimates
census_csv = "Choropleth Testing/Resources/county_populations_census_2010-17.csv"
county_pop = pd.read_csv(census_csv, encoding='latin-1')

# Rename columns
county_pop = county_pop.rename(columns = {'April 1, 2010 - Census': '2010 Census',
                                          'April 1, 2010 - Estimates Base': 'Estimate Base',
                                          'Population Estimate (as of July 1) - 2010': '2010 Estimate',
                                          'Population Estimate (as of July 1) - 2011': '2011 Estimate',
                                          'Population Estimate (as of July 1) - 2012': '2012 Estimate',
                                          'Population Estimate (as of July 1) - 2013': '2013 Estimate',
                                          'Population Estimate (as of July 1) - 2014': '2014 Estimate',
                                          'Population Estimate (as of July 1) - 2015': '2015 Estimate',
                                          'Population Estimate (as of July 1) - 2016': '2016 Estimate',
                                          'Population Estimate (as of July 1) - 2017': '2017 Estimate'})

# Parse geography column into county/state columns
county_pop["County"] = county_pop['Geography'].apply(lambda x: x.split(", ")[0].lower())
county_pop["County"] = county_pop["County"].apply(lambda x: x.replace(" county", ""))
county_pop["County"] = county_pop["County"].apply(lambda x: x.replace(" parish", ""))
county_pop["State"] = county_pop['Geography'].apply(lambda x: x.split(", ")[-1])
county_pop["FIPS"] = county_pop['Id'].apply(lambda x: x[-5:])
county_pop["State Abbreviation"] = county_pop['State'].apply(lambda x: reverse_state_abbrev[x])
county_pop["State"] = county_pop['State'].apply(lambda x: x.lower())

# Remove unnecessary cols
county_pop = county_pop[['FIPS','County','State','State Abbreviation','2010 Census','2011 Estimate','2012 Estimate','2013 Estimate','2014 Estimate','2015 Estimate','2016 Estimate','2017 Estimate']]

county_pop.head()

,FIPS,County,State,State Abbreviation,2010 Census,2011 Estimate,2012 Estimate,2013 Estimate,2014 Estimate,2015 Estimate,2016 Estimate,2017 Estimate
0,01001,autauga,alabama,AL,54571,55199,54927,54695,54864,54838,55278,55504
1,01003,baldwin,alabama,AL,182265,186534,190048,194736,199064,202863,207509,212628
2,01005,barbour,alabama,AL,27457,27351,27175,26947,26749,26264,25774,25270
3,01007,bibb,alabama,AL,22915,22745,22658,22503,22533,22561,22633,22668
4,01009,blount,alabama,AL,57322,57562,57595,57623,57546,57590,57562,58013


In [55]:
# Change county population data format to match other datasets (new rows for each year)?
county_pop_2010_2017 = []
year_cols = ['2010 Census','2011 Estimate','2012 Estimate','2013 Estimate','2014 Estimate','2015 Estimate','2016 Estimate','2017 Estimate']

for i in range(len(county_pop)):
    county_fips = county_pop.iloc[i,0]
    county_name = county_pop.iloc[i,1]
    county_state = county_pop.iloc[i,2]
    county_st_abbrev = county_pop.iloc[i,3]
    for j in range(4,len(county_pop.iloc[i])-1):
        year = year_cols[int(j-4)].split()[0]
        county_yr_pop = county_pop.iloc[i,j]
        county_yr_row = {'FIPS':county_fips,
                          'County':county_name,
                          'State':county_state,
                          'State Abbreviation':county_st_abbrev,
                          'Year':year,
                          'Population':county_yr_pop}
        county_pop_2010_2017.append(county_yr_row)
        
county_pop_2010_2017 = pd.DataFrame(county_pop_2010_2017)
county_pop_2010_2017.head()

,County,FIPS,Population,State,State Abbreviation,Year
0,autauga,01001,54571,alabama,AL,2010
1,autauga,01001,55199,alabama,AL,2011
2,autauga,01001,54927,alabama,AL,2012
3,autauga,01001,54695,alabama,AL,2013
4,autauga,01001,54864,alabama,AL,2014
